In [1]:
import os
import shutil
import sys
from tqdm import tqdm

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

tf.get_logger().setLevel('ERROR')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk

#for BERT
import transformers

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print(tf.__version__)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9420004554228447128
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 22723493888
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16656944599098317199
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6"
]
2.5.0


In [4]:
# GPU options to limit OOM erors
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Reload model

In [5]:
#RELOAD MODEL

saved_model_path = 'maturity_bert'

reloaded_model = tf.saved_model.load(saved_model_path)

In [6]:
reloaded_model

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x1fb978845b0>

## Label data subset for further round of active learning

In [7]:
complete = pd.read_csv('data/test_2021_maturity.csv', index_col=0)

In [8]:
#My version of the DF didn't have the 'feature' columns so made it here
#complete['feature'] = (complete.title + ' ' + complete.abstract)

In [11]:
complete.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 498 entries, 0 to 1432
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   feature  498 non-null    object
 1   truth    498 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 11.7+ KB


In [12]:
labeldf = complete[['feature']].copy()
labeldf.tail(10)

,feature
id,
1415,Novel Subtypes of Pulmonary Emphysema Based on...
1417,Machine learning algorithms can more efficient...
1419,An Effective Deep Network for Automatic Segmen...
1422,Prediction of pathologic complete response to ...
1423,Prediction of blood lactate values in critical...
1424,Artificial intelligence for prediction of COVI...
1428,"XGBoost, a Machine Learning Method, Predicts N..."
1429,Using machine learning methods to predict nonh...
1431,Machine learning model predicts short-term mor...


In [13]:
labeldf[labeldf['feature'].isna()]

,feature
id,


In [14]:
labeldf.dropna(subset=['feature'], inplace=True)
labeldf[labeldf['feature'].isna()]

,feature
id,


In [15]:
labeldf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 498 entries, 0 to 1432
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   feature  498 non-null    object
dtypes: object(1)
memory usage: 7.8+ KB


In [16]:
def label_unlabelled(df, feature_column = 'feature', model=reloaded_model, number_to_label='all', cpu_labelling=False, chunk_size=100):
    
    # Either use the whole df, random sample of size specified or a list of indices
    if number_to_label == 'all':
        labelled_df = df[feature_column].copy()
        
    elif isinstance(number_to_label, list):
        labelled_df = df.loc[number_to_label, feature_column].copy()
        
    else:
        assert isinstance(number_to_label, int), "Number to label must be 'all' or an integer subset to label"
        assert number_to_label < len(df), "When specifying a subset to label, must be less than the total number of samples"
        labelled_df = df[feature_column].sample(number_to_label).copy()
        
    # Add a column to the DF for labels
    labelled_df = labelled_df.to_frame()
    labelled_df['include'] = np.nan 
    
    # Decide what device we want TF to use
    if cpu_labelling:
        device = '/cpu:0'
        print("Labelling with CPU...")
    else:
        device = '/gpu:0'
        print("Labelling with GPU...")
     
    # Label by specified chunk size
    with tqdm(total=len(labelled_df), file=sys.stdout) as pbar:
        for chunk_i in range(0, len(labelled_df.index), chunk_size):
            
            chunk = labelled_df.index[chunk_i:chunk_i + chunk_size]
        
            try:
                with tf.device(device):
                    labels = tf.sigmoid(model(tf.constant(labelled_df.loc[chunk, 'feature'])))
                labelled_df.loc[chunk, 'include'] = labels
                pbar.update(len(chunk))
            except Exception as e:
                print(e)
                print("Returning (possibly) partially labelled dataset...")
                return labelled_df
                break
            
    return labelled_df

In [17]:
labelled = label_unlabelled(labeldf, number_to_label='all', cpu_labelling=False, chunk_size=50)

Labelling with GPU...
100%|████████████████████████████████████████████████████████████████████████████████| 498/498 [00:09<00:00, 53.30it/s]


In [15]:
uncertain = labelled[(labelled.include < 0.9) & (labelled.include > 0.1)]
uncertain.head(20)

,feature,include
4135,Rapid Artificial Intelligence Solutions in a P...,0.847937
9051,Evaluation of the Accuracy of a Video and AI S...,0.203166
11401,Microscope-Based Automated Quantification of L...,0.416859
11545,Deep learning diagnostic and risk-stratificati...,0.284014
14228,Handling missing MRI sequences in deep learnin...,0.126651
15039,Deep learning detection of subtle fractures us...,0.459525
25336,Transforming self-reported outcomes from a str...,0.594242
28168,Machine Learning-Based MRI Texture Analysis to...,0.720846
28940,Deep learning for automated sleep staging usin...,0.607522
29019,Cost-effectiveness of Autonomous Point-of-Care...,0.842223


In [16]:
labelled['include_rounded'] = np.round(labelled.include)

In [17]:
labelled.include_rounded.value_counts()

0.0    31275
1.0     1197
Name: include_rounded, dtype: int64

In [18]:
labelled.tail(5)

,feature,include,include_rounded
161510,On the use of neural network techniques to ana...,1.577470e-07,0.0
161511,Analysis of quantitative EEG with artificial n...,1.099111e-07,0.0
161513,Neural networks as a prognostic tool for patie...,1.232864e-06,0.0
161517,Automated interpretation of myocardial SPECT p...,2.712667e-07,0.0
161524,Acute pulmonary embolism: cost-effectiveness a...,6.019955e-07,0.0


In [19]:
labelled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32472 entries, 0 to 161524
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   feature          32472 non-null  object 
 1   include          32472 non-null  float64
 2   include_rounded  32472 non-null  float64
dtypes: float64(2), object(1)
memory usage: 2.0+ MB


In [20]:
complete.dropna(subset=['feature'], inplace=True)
complete.tail(5)

,pmid,doi,title,abstract,article_date,pubmed_date,article_type,lang,journal,journal_short,journal_country,authors,author_affils,keywords,mesh_terms,references_pmids,feature,include
161510,9438272,NaN,On the use of neural network techniques to ana...,This is the third communication on the use of ...,NaN,23/01/1998,Clinical Trial,eng,Neuropsychobiology,Neuropsychobiology,Switzerland,"['Baumgart-Schmitt R', 'Herrmann W M', 'Eilers...",NaN,NaN,"['Algorithms', 'Electroencephalography', 'Elec...",NaN,On the use of neural network techniques to ana...,1.0
161511,9438271,NaN,Analysis of quantitative EEG with artificial n...,Artificial neural networks (ANN) are widely us...,NaN,23/01/1998,Clinical Trial,eng,Neuropsychobiology,Neuropsychobiology,Switzerland,"['Winterer G', 'Ziller M', 'Klöppel B', 'Heinz...",NaN,NaN,"['Alcoholism', 'Algorithms', 'Discriminant Ana...",NaN,Analysis of quantitative EEG with artificial n...,1.0
161513,9436967,NaN,Neural networks as a prognostic tool for patie...,Patients with non-small cell carcinoma of the ...,NaN,22/01/1998,Journal Article,eng,Modern pathology : an official journal of the ...,Mod Pathol,United States,"['Bellotti M', 'Elsner B', 'Paez De Lima A', '...",NaN,NaN,"['Adenocarcinoma', 'Antigens, Nuclear', 'Bioma...",NaN,Neural networks as a prognostic tool for patie...,1.0
161517,9430460,NaN,Automated interpretation of myocardial SPECT p...,The purpose of this study was to develop a com...,NaN,16/01/1998,Comparative Study,eng,Journal of nuclear medicine : official publica...,J Nucl Med,United States,"['Lindahl D', 'Palmer J', 'Ohlsson M', 'Peters...",NaN,NaN,"['Coronary Angiography', 'Coronary Disease', '...",NaN,Automated interpretation of myocardial SPECT p...,1.0
161524,9423655,NaN,Acute pulmonary embolism: cost-effectiveness a...,To evaluate the cost-effectiveness of artifici...,NaN,10/01/1998,Journal Article,eng,Radiology,Radiology,United States,"['Tourassi G D', 'Floyd C E', 'Coleman R E']",NaN,NaN,"['Acute Disease', 'Angiography', 'Cost-Benefit...",NaN,Acute pulmonary embolism: cost-effectiveness a...,1.0


In [21]:
print(len(complete))
print(len(labelled))

32472
32472


In [22]:
complete['include_fuzzy'] = labelled['include']
complete['include'] = labelled['include_rounded']

In [23]:
# SAVE FINAL FILES

In [24]:
#uncertain.to_csv("final_outputs/comparative_uncertain.csv")

In [25]:
complete.to_csv("final_outputs/comparative_labelled_2.csv")

In [26]:
complete['include'].value_counts()

0.0    31275
1.0     1197
Name: include, dtype: int64